In [1]:
%pylab inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import numpy.random as rng
import pandas.io.data as web
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
class SmallConfig(object):
    """Small config."""
    input_size = 5
    output_size = 5
        #target_size = 1
    num_steps = 20
    num_layers = 2

    num_samples = 2
    learning_rate = 0.5
    hidden_size = 21

In [3]:
class Model():
    def __init__(self, config, training=True):
        #CONFIG
        input_size = self.input_size =  config.input_size
        output_size = self.output_size =  config.output_size
            #target_size = self.target_size =  config.target_size
        num_steps = self.num_steps =  config.num_steps
        
        num_samples = self.num_samples =  config.num_samples
        learning_rate = self.learning_rate =  config.learning_rate
        hidden_size = self.hidden_size =  config.hidden_size
        num_layers = self.num_layers =  config.num_layers


        # define placeholders 
        self.environment_ = tf.placeholder(tf.float32, [None, self.input_size])
            #self.targets_ = tf.placeholder(tf.float32, [None,  self.target_size]) #shape?

        #define cell
        cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=0.0)
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * config.num_layers)
        cell = tf.nn.rnn_cell.InputProjectionWrapper(cell,  self.input_size)
        cell = tf.nn.rnn_cell.OutputProjectionWrapper(cell, self.output_size)
        self.cell = cell

        #iterate through timesteps
        self.LSTM_iter()     #asign self.final_state and self.outputs
        
        self.outputs_softmax = tf.nn.softmax(self.outputs)   # softmax[i, j] = exp(logits[i, j]) / sum(exp(logits[i]))
        
        self.sample_policy()
        
        self.get_status()
        self.get_reward()

        ones = tf.ones_like(self.training_target_cols)
        gradient_ = tf.nn.sigmoid_cross_entropy_with_logits(self.training_target_cols, ones)
        self.gradient = tf.transpose(tf.reshape(gradient_, [-1,2,num_samples]), [0,2,1]) #[?,5,2]
        
            #self.reward = tf.reduce_mean(self.targets_)
        
        self.cost = tf.mul(self.gradient , tf.expand_dims(self.reward, -1))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.cost)

        
    def LSTM_iter(self):
        self.outputs=[]
        self.initial_state = self.cell.zero_state(1, tf.float32)    #batch size?
        self.state = self.initial_state
            
        with tf.variable_scope("RNN"):
            for time_step in range(self.num_steps):   #####num_steps???
                if time_step > 0: tf.get_variable_scope().reuse_variables()      #whats this?
                inp = self.environment_[time_step,:]
                inp = tf.reshape(inp, [1,-1])
                (cell_output, self.state) = self.cell(inp, self.state)
                self.outputs.append(cell_output) 

        self.final_state = self.state 
        self.outputs = tf.reshape(tf.concat(1, self.outputs), [-1, self.output_size])
        
    def sample_policy(self):
        self.sample = tf.multinomial(tf.log(self.outputs), self.num_samples)
        
        relevant_target_column = {}
        for sample_iter in range(self.num_samples):
            sample_n = self.sample[:,sample_iter]
            sample_mask = tf.cast(tf.reshape(tf.one_hot(sample_n, self.output_size), 
                                             [-1,self.output_size]), float32)
            relevant_target_column[sample_iter] = tf.reduce_sum(
                                                self.outputs_softmax * sample_mask,1)
        self.training_target_cols = tf.concat(1, [tf.reshape(t, [-1,1]) for t in relevant_target_column.values()])
        
    def get_status(self):
        self.status = None
        return
    def get_reward(self):
        self.reward = None
        return

In [4]:
sess = tf.InteractiveSession()
# initialize variables to random values
with tf.variable_scope("model", reuse=None):
    m = Model(config = SmallConfig)
with tf.variable_scope("model", reuse=True):
    mvalid = Model(config = SmallConfig, training=False)
sess.run(tf.initialize_all_variables())


In [5]:
inputs = np.random.random([20,5])
target = 2*(np.random.random([1,1]) - .5)

In [11]:
for epoch in range(200):
    sess.run(m.optimizer, feed_dict={m.environment_: inputs, m.targets_: target})
    if np.sqrt(epoch+1)%1== 0:
        c = sess.run([ mvalid.cost], feed_dict={mvalid.environment_: inputs, mvalid.targets_: target})
        c = np.mean(c)
        print("Epoch:", '%04d' % (epoch+1), "cost=",c, )

Epoch: 0001 cost= -0.190774
Epoch: 0004 cost= -0.192111
Epoch: 0009 cost= -0.192357
Epoch: 0016 cost= -0.190586
Epoch: 0025 cost= -0.191192
Epoch: 0036 cost= -0.192431
Epoch: 0049 cost= -0.192226
Epoch: 0064 cost= -0.192254
Epoch: 0081 cost= -0.192042
Epoch: 0100 cost= -0.192146
Epoch: 0121 cost= -0.190722
Epoch: 0144 cost= -0.191567
Epoch: 0169 cost= -0.191888
Epoch: 0196 cost= -0.191204


In [2]:
def make_class(arg1, arg2, arg3):
    class MyClass(object):
        thing = arg1
        thing2=arg2
        thing3=arg3
    return MyClass


In [3]:
TheClass = make_class(1,2,3)

In [4]:
TheClass.thing

1

In [13]:
import os
import pickle as cPickle
with open('dumbfile.pkl', 'rb') as f:
            saved_model_args = cPickle.load(f)

In [11]:
with open('dumbfile.pkl', 'wb') as f:
    cPickle.dump(AnotherClass, f)

In [10]:
class AnotherClass(object):
    fuck = 1
    shit = 2

In [15]:
saved_model_args.shit

2

In [18]:
from lxml import html
import requests

In [25]:
page = requests.get('https://news.ycombinator.com/news')
tree = html.fromstring(page.text)


In [27]:
tree?

In [24]:
page

AttributeError: 'HtmlElement' object has no attribute 'children'

In [28]:
import urllib.request as urllib
from lxml import html

In [33]:
url = 'https://news.ycombinator.com/news'
page = html.fromstring(urllib.request.urlopen(url).read())


In [67]:
def find_links(page, keyword, action = print):
    for link in page.xpath("//a"):
        if link.text:
            if keyword in link.text:
                action(keyword)
        #print("Name", link.text, "URL", link.get("href"))

In [69]:
find_links(page, 'the', action = print)

the
the
the
the
the


In [53]:
def dumbaction(keyword):
    print('keyword')